In [1]:
%load_ext autoreload
%autoreload
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>
    div.output_area{
        max-height:10000px;
        overflow:scroll;
    }
</style>"""))

import shapefile
import pygeohash as geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
from geoUtils import *
import string
letters = string.ascii_uppercase
basedir="/Users/tgadf/Downloads/osm"

Notebook Last Run Initiated: 2018-09-19 19:26:08.387067


In [3]:
import pickle
prec=7
from glob import glob
from os.path import basename,splitext, join
from collections import Counter

#mtypes   = set([splitext(basename(x))[0].split('-')[1] for x in vals])
mtypes  = ["traffic", "pofw", "pois", "transport", "buildings"]
mtypes  = ["buildings"]
mtypes  = ["pofw"]
mtypes  = ["traffic"]
mtypes  = ["transport"]
mtypes  = ["pois"]
mtypes  = ["traffic", "pofw", "pois", "transport"]

print("Mtypes --> {0}".format(mtypes))
for mtype in mtypes:
    if mtype in ["pofw","traffic","transport","pois"]:
        vals    = glob(join(basedir, "*-{0}-{1}-geos.p".format(mtype, prec)))        
        states  = set([splitext(basename(x))[0].split('-{0}'.format(mtype))[0] for x in vals])
    else:
        vals    = glob(join(basedir, "*-{0}-geos.p".format(prec)))
        states  = set([splitext(basename(x))[0].split('-gis_osm')[0] for x in vals])
    print("States --> {0}".format(states))

    print("============== {0} ==============".format(mtype))
    fnames = letters[:1]
    for let in range(len(fnames)):
        ngeosMtypes = 0
        ngeos = 0
        geomap  = {}
        records = {}
        cntr    = Counter()
        for istate,state in enumerate(states):
            if istate % len(fnames) != let:
                continue
            base = "{0}*{1}*{2}".format(state, mtype, prec)
            recfiles = glob(join(basedir, "{0}-data.p".format(base)))
            geofiles = glob(join(basedir, "{0}-geos.p".format(base)))

            #print("  Found {0} and {1} rec/geo files for {2}".format(len(recfiles), len(geofiles), mtype))
            for i in range(len(geofiles)):
                recfile = recfiles[i]
                geofile = geofiles[i]

                try:
                    recdata = pickle.load(open(recfile, "rb"))
                    geodata = pickle.load(open(geofile, "rb"))
                    #print("Opened {0}".format(recfile))
                    #print("Opened {0}".format(geofile))
                except:
                    print("Could not open {0}".format(geofile))
                    1/0
                    continue

                #print(geodata.keys())
                #continue
                ngeos = 0
                for geoid,geos in geodata.items():
                    ngeos += len(geos)
                    ngeosMtypes += len(geos)
                    for geo in geos:
                        if geomap.get(geo) is None:
                            geomap[geo] = Counter()
                        #print(geo,mtype,geoid)
                        #print(geoid)
                        #print(recdata.keys())
                        if recdata.get(geoid) is not None:
                            try:
                                #print(geoid)
                                #print(recdata[geoid])
                                #name = recdata[geoid]['Name']
                                #print(name)
                                name = geoid
                            except:
                                name = geoid
                                #print(name)
                        else:
                            name = geoid
                        cntr[name] += 1
                        geomap[geo][name] += 1

                for geoid,rec in recdata.items():
                    records[geoid] = rec['Name']
                    break

            print("{0: <20}\t{1: <20}\t{2: <12}\t{3: <12}\t---> {4} <---".format(mtype,state,ngeos,ngeosMtypes,len(geomap)))            
        print("{0: <20}\t{1: <20}\t{2: <12}\t{3: <12}\t---> {4} <---".format(mtype,"","",ngeosMtypes,len(geomap)))

        for k,v in cntr.most_common():
            k = str(k)
            if k.find(" ") != -1 or k.find(";") != -1 or k.find("_") != -1 or k.find(",") != -1 or k[0].isupper():
                print("buildingConv[\"{0}\"] = \"{1}\"".format(k, k))
                found = True
                    
        for geo in geomap.keys():
            geomap[geo] = dict(geomap[geo])
        

        fname="geomap-{0}-{1}OSM.p".format(prec, mtype) #, fnames[let])
        print("Writing {0}".format(fname))
        pickle.dump(geomap,  open(fname, "wb"))

        fname="georec-{0}-{1}OSM.p".format(prec, mtype) #, fnames[let])
        print("Writing {0}".format(fname))
        pickle.dump(records,  open(fname, "wb"))

Mtypes --> ['traffic', 'pofw', 'pois', 'transport']
States --> {'indiana', 'alabama', 'arkansas', 'kansas', 'south-dakota', 'idaho', 'washington', 'alaska', 'pennsylvania', 'florida', 'arizona', 'south-carolina', 'wisconsin', 'texas', 'louisiana', 'hawaii', 'massachusetts', 'maine', 'new-jersey', 'georgia', 'nebraska', 'illinois', 'virginia', 'new-hampshire', 'west-virginia', 'new-york', 'montana', 'utah', 'connecticut', 'iowa', 'colorado', 'michigan', 'missouri', 'kentucky', 'mississippi', 'tennessee', 'delaware', 'north-dakota', 'new-mexico', 'oklahoma', 'ohio', 'puerto-rico', 'rhode-island', 'maryland', 'oregon', 'california', 'district-of-columbia', 'nevada', 'vermont', 'wyoming', 'minnesota'}
============== traffic ==============
traffic             	indiana             	23471       	23471       	---> 23153 <---
traffic             	alabama             	9517        	32988       	---> 32485 <---
traffic             	arkansas            	3406        	36394       	---> 35836 <---
tra

pofw                	colorado            	2165        	160436      	---> 160083 <---
pofw                	michigan            	6888        	167324      	---> 166959 <---
pofw                	missouri            	7096        	174420      	---> 174027 <---
pofw                	kentucky            	6461        	180881      	---> 180433 <---
pofw                	mississippi         	8784        	189665      	---> 189195 <---
pofw                	tennessee           	10756       	200421      	---> 199887 <---
pofw                	delaware            	260         	200681      	---> 200146 <---
pofw                	north-dakota        	813         	201494      	---> 200950 <---
pofw                	new-mexico          	1295        	202789      	---> 202239 <---
pofw                	oklahoma            	5062        	207851      	---> 207295 <---
pofw                	north-carolina      	10383       	218234      	---> 217630 <---
pofw                	ohio                	12858       	231092    

transport           	massachusetts       	8420        	41774       	---> 41359 <---
transport           	maine               	153         	41927       	---> 41509 <---
transport           	new-jersey          	949         	42876       	---> 42432 <---
transport           	georgia             	961         	43837       	---> 43381 <---
transport           	nebraska            	151         	43988       	---> 43532 <---
transport           	illinois            	8049        	52037       	---> 51394 <---
transport           	virginia            	1903        	53940       	---> 53275 <---
transport           	new-hampshire       	659         	54599       	---> 53926 <---
transport           	west-virginia       	100         	54699       	---> 54023 <---
transport           	new-york            	6190        	60889       	---> 59923 <---
transport           	montana             	69          	60958       	---> 59992 <---
transport           	utah                	730         	61688       	---> 607

In [7]:
import pickle
data = pickle.load(open("geomap-7-buildingsOSMA.p", "rb"))
#import pandas as pd
#pddf = pd.DataFrame(data)

In [8]:
data

{'dp4dpq4': {'stadium': 1, 'postoffice': 1},
 'dp4dpmc': {'stadium': 1},
 'dp4dpmb': {'stadium': 1, 'building': 1},
 'dp4dpq0': {'stadium': 1, 'building': 1},
 'dp4dpq6': {'stadium': 1, 'building': 1, 'postoffice': 1},
 'dp4dpq3': {'stadium': 1},
 'dp5kb31': {'stadium': 1, 'building': 1},
 'dp5kb30': {'stadium': 1, 'building': 1},
 'dp5kb2c': {'stadium': 1},
 'dnfqzhf': {'stadium': 1},
 'dnfqzhd': {'stadium': 1},
 'dnfqzh9': {'stadium': 1},
 'dnfqzhc': {'stadium': 1},
 'dp4g0rr': {'stadium': 1, 'building': 1},
 'dp4dpqx': {'stadium': 1, 'building': 1},
 'dp4dpqy': {'stadium': 1, 'building': 1},
 'dp4dpwb': {'stadium': 1, 'building': 1},
 'dp4dpqz': {'stadium': 1, 'building': 1},
 'dp4dpqw': {'stadium': 1, 'building': 1},
 'dp73qmn': {'building': 1},
 'dp73qmq': {'building': 1},
 'dp73qmp': {'building': 1},
 'dngwsqe': {'building': 1},
 'dngwsqs': {'building': 1},
 'dnfqxz6': {'building': 1},
 'dnfqxz3': {'building': 1},
 'dnfqxz4': {'building': 1},
 'dnfqxz1': {'building': 1},
 'dnfqxx

In [5]:
import pickle
data = pickle.load(open("/Users/tgadf/Downloads/osm/wyoming-gis_osm_buildings_a_free_1-7-data.p", "rb"))

In [5]:
data

{'dn7jy9x': {'CollegesUniversities': 'CollegesUniversities'},
 'dre6znp': {'CollegesUniversities': 'CollegesUniversities'},
 'dpqhus0': {'CollegesUniversities': 'CollegesUniversities'},
 'dr4vh9n': {'CollegesUniversities': 'CollegesUniversities'},
 '9t9p778': {'CollegesUniversities': 'CollegesUniversities'},
 '9vzbh65': {'CollegesUniversities': 'CollegesUniversities'},
 'dr95f7f': {'CollegesUniversities': 'CollegesUniversities'},
 'dhwggff': {'CollegesUniversities': 'CollegesUniversities'},
 '9q5csvt': {'CollegesUniversities': 'CollegesUniversities'},
 '9qh4jqn': {'CollegesUniversities': 'CollegesUniversities'},
 'drt35bj': {'CollegesUniversities': 'CollegesUniversities'},
 '9q9kf24': {'CollegesUniversities': 'CollegesUniversities'},
 'dp3wfzy': {'CollegesUniversities': 'CollegesUniversities'},
 'dpnegjk': {'CollegesUniversities': 'CollegesUniversities'},
 '9wz08bm': {'CollegesUniversities': 'CollegesUniversities'},
 '9q9pm2s': {'CollegesUniversities': 'CollegesUniversities'},
 '9q5bry

In [5]:
files

['geomap-7-buildingsOSM.p',
 'geomap-7-Car.p',
 'geomap-7-College.p',
 'geomap-7-Commerce.p',
 'geomap-7-HERE.p',
 'geomap-7-Industry.p',
 'geomap-7-loc.p',
 'geomap-7-pofwOSM.p',
 'geomap-7-poisOSM.p',
 'geomap-7-Rail.p',
 'geomap-7-Road.p',
 'geomap-7-Roads.p',
 'geomap-7-RoadsA.p',
 'geomap-7-RoadsB.p',
 'geomap-7-RoadsC.p',
 'geomap-7-RoadsD.p',
 'geomap-7-RoadsE.p',
 'geomap-7-Schools.p',
 'geomap-7-Terminal.p',
 'geomap-7-Terminals.p',
 'geomap-7-trafficOSM.p',
 'geomap-7-transportOSM.p',
 'geomap-7-Venues.p']